# LUCAS – Result plots

<div class="alert alert-block alert-info">
        This notebook refers to the studies presented in <b>Chapter 4</b> of the Ph.D. thesis [4].
    With this notebook, you can process the raw LUCAS 2012 soil dataset to work with the 1D CNNs [2].
    We can not guarantee completeness or correctness of the code.
    If you find bugs or if you have suggestions on how to improve the code, we encourage you to post your ideas as <a href="https://github.com/felixriese/lucas-processing/issues">GitHub issue</a>.
</div>

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

import lucas_utils as utils

In [ ]:
dataset = utils.get_data(
    targetvar="superclass",
    path="data/5_lucas_final_",
    scaling_x=False,
    scaling_y=False)

classes = ['super_L', 'super_S', 'super_T', 'super_U']

y_train = label_binarize(dataset[3], classes=classes)
y_val = label_binarize(dataset[4], classes=classes)
y_test = label_binarize(dataset[5], classes=classes)

print(y_val.shape, y_test.shape)

In [ ]:
# CHANGE "val" or "test"
MODE = "test"

## Confusion matrices

In [ ]:
fig, axlist = plt.subplots(3,2, figsize=(11,16), sharex=True, sharey=True)
axlist = axlist.flatten()
fontsize = 15

runs = [
    "20200318_151210_RF",
    "20200318_151433_LiuEtAl",
    "20200318_150728_HuEtAl",
    "20200318_145323_LucasCNN",
    "20200318_132141_LucasResNet",
    "20200318_175316_LucasCoordConv",
]

names = {
    "LucasCNN": "LucasCNN",
    "LucasResNet": "LucasResNet",
    "LucasCoordConv": "LucasCoordConv",
    "HuEtAl": "Hu et al. (2015)",
    "LiuEtAl": "Liu et al. (2018)",
    "RF": "RF"
}

for i, run in enumerate(runs):
    # get y_true and y_pred
    if MODE == "val":
        y_true = y_val
        y_pred = pickle.load(open("results/"+run+"_yvalpred.p", "rb"))
    elif MODE == "test":
        y_true = y_test
        y_pred = pickle.load(open("results/"+run+"_ytestpred.p", "rb"))

    # Compute confusion matrix
    if not "RF" in run:
        cnf_matrix = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    else:
        cnf_matrix = confusion_matrix(y_true.argmax(axis=1), y_pred)
    
    # Plot
    img = utils.plot_confusion_matrix(
        cnf_matrix, [c[6:] for c in classes],
        axlist[i], normalize=True, title=names[run[16:]],
        fontsize=fontsize,
        show_xlabel=bool(i>3),
        show_ylabel=not bool(i%2))
                                

# plt.tight_layout()

# colorbar
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.87, 0.15, 0.02, 0.70])  # [left, bottom, width, height]
cbar = fig.colorbar(img, cax=cbar_ax)
cbar_ax.tick_params(labelsize=fontsize)
cbar.set_label(label=r'$\frac{\mathrm{Number\ of\ classified\ datapoints\ as\ class\ }x}{\mathrm{Number\ of\ datapoints\ in\ class\ }x}$',
               fontsize=20,
               labelpad=20)

# save and show
plt.savefig("plots/confusion_"+MODE+"_portrait.pdf", bbox_inches='tight')
plt.show()

## Histograms

In [ ]:
print("Train:", len(y_train_numeric))
print("Val:", len(y_val_numeric))
print("Test:", len(y_test_numeric))

In [ ]:
fontsize = 15
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 4), sharey=True)

class_dict = {"super_L": 0, 'super_S': 1, 'super_T': 2, 'super_U': 3}

y_train_numeric = [class_dict[x] for x in list(dataset[3])]
y_val_numeric = [class_dict[x] for x in list(dataset[4])]
y_test_numeric = [class_dict[x] for x in list(dataset[5])]

print(np.unique(y_train_numeric, return_counts=True))


bins = [-0.5,0.5, 1.5, 2.5, 3.5]

n, _, patches = ax1.hist(
    y_train_numeric, label="Training", color="#4664aa", bins=bins, 
    alpha=0.8, histtype="bar")
print("Max:", np.max(n))

ax2.hist(
    y_val_numeric, bins=bins, label="Validation",
    histtype="bar", linewidth=3, color="#e58323")
ax3.hist(
    y_test_numeric, bins=bins, label="Test",
    histtype="bar", linewidth=3, color="#9b1723")

# set titles:
ax1.set_title("Training", y=0.88, fontsize=fontsize)
ax2.set_title("Validation", y=0.88, fontsize=fontsize)
ax3.set_title("Test", y=0.88, fontsize=fontsize)

for ax in (ax1, ax2, ax3):
    ax.set_xlabel("Soil classes", fontsize=fontsize, labelpad=10)
    ax.set_xticks([0,1,2,3])
    ax.set_xticklabels([c[-1] for c in classes], fontsize=fontsize)
    ax.tick_params(axis="y", labelsize=fontsize)
    ax.set_ylim(0, 4000)
    ax.xaxis.grid(alpha=0.3)
    ax.yaxis.grid(alpha=0.3)

ax1.set_ylabel("Number of datapoints", fontsize=fontsize, labelpad=10)

plt.tight_layout()

plt.savefig("plots/class_3histos.pdf", bbox_inches='tight')

## False classification plot

In [ ]:
y_test_sand = utils.get_data(targetvar="sand", path="data/5_lucas_final_")[5]
y_test_silt = utils.get_data(targetvar="silt", path="data/5_lucas_final_")[5]
y_test_clay = utils.get_data(targetvar="clay", path="data/5_lucas_final_")[5]
y_test_superclass = utils.get_data(targetvar="superclass", path="data/5_lucas_final_")[5]

color = {"super_L": 2, "super_S": 1, "super_T": 4, "super_U": 3}
classes = list(color.keys())
fontsize = 15
def get_color(class_list):
    return [int(color[cl]) for cl in class_list]

# colormap
cmap = plt.cm.viridis
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm', cmaplist, cmap.N)
bounds = np.linspace(1, 5, 5)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

fig, axlist = plt.subplots(3,2, figsize=(11,16), sharex=True, sharey=True)
axlist = axlist.flatten()
fontsize = 15

for i, run in enumerate(runs):
    ax = axlist[i]

    y_pred = pickle.load(open("results/"+run+"_ytestpred.p", "rb"))
    if y_pred.shape != (3208, ):
        y_pred = np.argmax(y_pred, axis=1)

    # true_index = np.argmax(y_test, axis=1) == np.argmax(y_pred, axis=1)
    false_index = np.argmax(y_test, axis=1) != y_pred

    ax.scatter(y_test_clay[false_index], y_test_silt[false_index],
                c=get_color(y_test_superclass[false_index]),
                marker="o", alpha=0.3, cmap=cmap, norm=norm, label="")
    
    ax.set_title(names[run[16:]], fontsize=fontsize, weight="bold")
    ax.set_xlim(0,100)
    ax.set_ylim(0,100)
    if i > 3:
        ax.set_xlabel("Clay content in %", fontsize=fontsize, labelpad=15)
    if i % 2 == 0:
        ax.set_ylabel("Silt content in %", fontsize=fontsize)
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(fontsize)
    
    # add legend
    if i % 2 != 0:
        # plotting empty lists for legend
        for c in color.keys():
            ax.scatter(
                [], [],
                c="k",
                label=c[-1])
        
        ax.legend(fontsize=fontsize, title="Original soil class:", frameon=False)
        legend = ax.get_legend()
        legend.get_title().set_fontsize(fontsize)
        legend._legend_box.align = "left"
        for i, c in enumerate(color.keys()):
            legend.legendHandles[i].set_color(cmap(norm(color[c])))
plt.tight_layout()
plt.savefig("plots/wrong_classifications.pdf", bbox_inches='tight')       